## Problems

### 34-1 Independent set

> An __*independent set*__ of a graph $G = (V, E)$ is a subset $V' \subseteq V$ of vertices such that each edge in $E$ is incident on at most one vertex in $V'$. The __*independent-set problem*__ is to find a maximum-size independent set in $G$.

> __*a*__. Formulate a related decision problem for the independent-set problem, and prove that it is NP-complete. (Hint: Reduce from the clique problem.)

__Desicision problem:__

The independent set in $G$ is at most $k$.

__ISP $\in$ NP__:

The certificate is a set of vertices $V'$. To verify the certificate, for each $u, v \in V'$, we should verify that $(u, v) \notin E$.

__ISP is NP-hard to CLIQUE $\Rightarrow$ ISP $\in$ NPC:__

_Construction:_

Suppose the complement graph of $G$ is $G'$, then $G$ for CLIQUE is equivalent to $G'$ for ISP.

_CLIQUE $\Rightarrow$ ISP:_

Suppose there exists a clique $C$ of size $k$ in $G$, we know $\forall u, v \in C$, $(u, v) \in E$. Therefore in the complement graph $G'$, $\forall u, v \in C$, $(u, v) \notin E$.

_ISP $\Rightarrow$ CLIQUE:_

Suppose there exists an independent-set $C$ of size $k$ in $G'$, we know $\forall u, v \in C$, $(u, v) \notin E$. Therefore in the graph $G$, $\forall u, v \in C$, $(u, v) \in E$.

> __*b*__. Suppose that you are given a "black-box" subroutine to solve the decision problem you defined in part (a). Give an algorithm to find an independent set of maximum size. The running time of your algorithm should be polynomial in $|V|$ and $|E|$, counting queries to the black box as a single step.

```
FOR k = |V| .. 1
    IF ISP(G, k)
    THEN RETURN k
RETURN 0
```

> Although the independent-set decision problem is NP-complete, certain special cases are polynomial-time solvable.

> __*c*__. Give an efficient algorithm to solve the independent-set problem when each vertex in $G$ has degree 2. Analyze the running time, and prove that your algorithm works correctly.

The graph is a set of chains and simple cycles.

In [1]:
def isp_in_deg_2(graph):
    n = len(graph)
    color = [0 for _ in range(n)]

    def search(u):
        if color[u] != 0:
            return
        color[u] = 1
        for v in graph[u]:
            if color[v] == 1:
                color[u] = -1
                break
        for v in graph[u]:
            search(v)

    for u in range(n):
        if len(graph[u]) == 1:
            search(u)
    for u in range(n):
        if color[u] == 0:
            search(u)

    return len(filter(lambda x: x == 1, color))

Runs in $O(|V||E|)$ time.